In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_infrastructure_damaged.csv
/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_casualties_daily.csv
/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_press_killed_in_gaza.csv
/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_west_bank_daily.csv


# 📊 Humanitarian Impact Dashboard – Gaza & West Bank (2023–2025)

---

## 🔴 Gaza Daily Casualties
> A time-series line chart showing daily reported deaths and injuries in Gaza.

- 🔹 Dataset: `cleaned_casualties_daily.csv`
- 🔹 Measures: `killed`, `ext_injured`
- 🔹 Visual: Dual-line Plotly chart (Apple-style red & orange)


In [9]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load dataset
gaza = pd.read_csv('/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_casualties_daily.csv')
gaza['report_date'] = pd.to_datetime(gaza['report_date'])

# Apple aesthetic fonts & colours
apple_font = "San Francisco, -apple-system, system-ui"
apple_red = "#FF3B30"
apple_orange = "#FF9500"

# Create figure
fig = go.Figure()

# Add killed line
fig.add_trace(go.Scatter(
    x=gaza['report_date'],
    y=gaza['killed'],
    mode='lines',
    name='Killed',
    line=dict(color=apple_red, width=2.5),
    hovertemplate='Killed: %{y}<br>Date: %{x|%b %d, %Y}',
))

# Add injured line
fig.add_trace(go.Scatter(
    x=gaza['report_date'],
    y=gaza['ext_injured'],
    mode='lines',
    name='Injured',
    line=dict(color=apple_orange, width=2.5),
    hovertemplate='Injured: %{y}<br>Date: %{x|%b %d, %Y}',
))

# Layout styling
fig.update_layout(
    title='Gaza Daily Casualties',
    title_font=dict(size=22, family=apple_font, color='black'),
    font=dict(family=apple_font, size=14),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=40, r=30, t=60, b=40),
    hovermode='x unified',
    legend=dict(x=0.85, y=1.05, bgcolor='rgba(0,0,0,0)', font=dict(size=13))
)

# Remove grid lines, Apple-style
fig.update_xaxes(showgrid=False, linecolor='lightgray')
fig.update_yaxes(showgrid=False, linecolor='lightgray')

fig.show()

---

## 🟥 West Bank Daily Fatalities
> A visualisation of verified killings per day in the West Bank region.

- 🔹 Dataset: `cleaned_west_bank_daily.csv`
- 🔹 Measures: `verified.killed`
- 🔹 Visual: Line chart, Apple-style dark red


In [10]:
# Load West Bank dataset
west_bank = pd.read_csv('/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_west_bank_daily.csv')
west_bank['report_date'] = pd.to_datetime(west_bank['report_date'])

# Apple aesthetic values
apple_font = "San Francisco, -apple-system, system-ui"
apple_dark_red = "#8B0000"

# Create figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=west_bank['report_date'],
    y=west_bank['verified.killed'],
    mode='lines',
    name='Fatalities',
    line=dict(color=apple_dark_red, width=2.5),
    hovertemplate='Killed: %{y}<br>Date: %{x|%b %d, %Y}',
))

# Layout styling
fig.update_layout(
    title='West Bank Daily Fatalities',
    title_font=dict(size=22, family=apple_font, color='black'),
    font=dict(family=apple_font, size=14),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=40, r=30, t=60, b=40),
    hovermode='x unified',
    legend=dict(x=0.85, y=1.05, bgcolor='rgba(0,0,0,0)', font=dict(size=13))
)

# Clean axes
fig.update_xaxes(showgrid=False, linecolor='lightgray')
fig.update_yaxes(showgrid=False, linecolor='lightgray')

fig.show()

---

## 📰 Journalist Fatalities by Role
> Classification of journalists killed during the conflict, based on role.

- 🔹 Dataset: `cleaned_press_killed_in_gaza.csv`
- 🔹 Extracted from: `notes` column
- 🔹 Visual: Horizontal bar chart (Apple grey tones)


In [13]:
press = pd.read_csv('/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_press_killed_in_gaza.csv')
press['notes'].head(10)

0    Al-Salhi, a photojournalist working for the Fo...
1    Ibrahim Mohammad Lafi, a photographer for Ain ...
2    Jarghoun, a journalist with Smart Media, was s...
3    Shamlakh, a freelance journalist, was killed a...
4    Al-Taweel, editor-in-chief of the Al-Khamsa Ne...
5    Alnwajha, a journalist with Khabar news agency...
6                                Details not available
7                                Details not available
8                                Details not available
9                                Details not available
Name: notes, dtype: object

In [14]:
import pandas as pd
import plotly.express as px

# Load press dataset
press = pd.read_csv('/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_press_killed_in_gaza.csv')

# We'll group by similar profession keyword (extracted from notes)
keywords = ['photojournalist', 'editor', 'freelance', 'journalist', 'correspondent', 'reporter', 'camera', 'videographer']
press['role'] = press['notes'].str.lower().apply(lambda note: next((k for k in keywords if k in note), 'other') if isinstance(note, str) else 'unknown')

# Count per role
role_counts = press['role'].value_counts().reset_index()
role_counts.columns = ['Role', 'Number']

# Plot Apple-style horizontal bar chart
fig = px.bar(
    role_counts.sort_values(by='Number', ascending=True),
    x='Number',
    y='Role',
    orientation='h',
    title='Journalist Fatalities by Role',
    color_discrete_sequence=['gray']
)

# Style to Apple aesthetic
fig.update_layout(
    font=dict(family="San Francisco, -apple-system, system-ui", size=14),
    title_font=dict(size=22, family="San Francisco, -apple-system, system-ui", color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=40, r=30, t=60, b=40),
    showlegend=False
)

fig.update_xaxes(showgrid=False, linecolor='lightgray')
fig.update_yaxes(showgrid=False, linecolor='lightgray')

fig.show()

---

## 🏚️ Infrastructure Damage (Log-Scaled Treemap)
> Aggregated structural damage grouped by sector with log-based sizing to enhance visibility.

- 🔹 Dataset: `cleaned_infrastructure_damaged.csv`
- 🔹 Groups: Residential, Education, Worship, Civic
- 🔹 Visual: Treemap (Apple-style blue gradient)


In [18]:
import pandas as pd
import numpy as np
import plotly.express as px

# Load dataset
infra = pd.read_csv('/kaggle/input/humanitarian-impact-gaza-westbank/cleaned_infrastructure_damaged.csv')

# Drop the date column
infra_summary = infra.drop(columns=['report_date'])

# Map to categories
category_map = {
    'civic_buildings': 'Civic',
    'educational_buildings': 'Education',
    'places_of_worship': 'Places of Worship',
    'residential': 'Residential'
}

grouped = {}

for col in infra_summary.columns:
    for keyword in category_map:
        if keyword in col:
            group = category_map[keyword]
            grouped[group] = grouped.get(group, 0) + infra_summary[col].sum()

# Convert to DataFrame with log-transformed values
tree_df = pd.DataFrame(list(grouped.items()), columns=['Sector', 'Raw Damage'])
tree_df['Log Damage'] = np.log1p(tree_df['Raw Damage'])  # log1p to avoid log(0)

# Plot Treemap using Log scale
fig = px.treemap(
    tree_df,
    path=['Sector'],
    values='Log Damage',
    title='Infrastructure Damage by Sector (Log-Scaled)',
    color='Log Damage',
    hover_data={'Raw Damage': True, 'Log Damage': False},
    color_continuous_scale='Blues'
)

# Apple-style
fig.update_layout(
    font=dict(family="San Francisco, -apple-system, system-ui", size=14),
    title_font=dict(size=22, family="San Francisco, -apple-system, system-ui", color='black'),
    margin=dict(l=40, r=40, t=60, b=40)
)

fig.show()

Between October 2023 and April 2025, the humanitarian impact on Gaza and the West Bank has been both extensive and alarming. Data-driven visualisations show key trends:

- **Gaza** faced intense daily violence with fatalities peaking sharply in the early months. Injuries followed a similar trend, with repeated spikes that coincided with escalation waves.
- In the **West Bank**, verified fatality rates were lower but sustained over time, indicating persistent tension across the territory.
- A notable portion of casualties were **press members**, with journalist roles ranging from photojournalists to editors. This highlights media exposure during the crisis.
- **Residential infrastructure** suffered the greatest damage, with **education and worship sites** also significantly affected. A logarithmic treemap reveals all sectors meaningfully while respecting the disproportionate scale of destruction.

This visual analysis presents a consolidated, unbiased representation of the humanitarian toll — enabling informed discussions and transparent documentation for advocacy and global reporting.
